## Initializng

In [1]:
import os
from pyspark.sql import SparkSession

In [4]:
import pyspark.sql.functions as F

In [5]:
jdbc_path = "/home/jovyan/drivers/postgresql-42.7.7.jar"
jdbc_url = "jdbc:postgresql://rc1a-h9rlcjswmfcyn9g0.mdb.yandexcloud.net:6432/dev_users_tool"
os.environ["JAVA_HOME"] = "/home/jovyan/jdk-17.0.12+7/"
creds = {
    "user": "tech_fr_pl_dev_admin",
    "password": "wh2MIUPFxCtH",
    "driver": "org.postgresql.Driver"
}

In [ ]:
spark = SparkSession.builder.appName("test").config("spark.driver.extraClassPath", jdbc_path).getOrCreate()

In [36]:
def read_data(table_name: str):
    return spark.read.jdbc(url=jdbc_url, table="params."+table_name, properties=creds)
def write_data(dataframe, table_name: str):
    return dataframe.write.jdbc(url=jdbc_url, mode="append", table="params."+table_name,properties=creds)

In [9]:
cluster_set_df = read_data("cluster_types_set")
priority_df = read_data("priority")

In [22]:
property_df = read_data("property")

In [23]:
cluster_df = read_data("cluster")

In [11]:
priority_start_ids_df = priority_df.select("cluster_types_set_id").distinct()

## Recursive cte

In [14]:
anchor_df = cluster_set_df.join(
    priority_start_ids_df,
    cluster_set_df.id == priority_start_ids_df.cluster_types_set_id,
    "semi"
).withColumn("root", F.col("id")) # Add the 'root' column

In [ ]:
final_results_df = anchor_df
iteration_df = anchor_df
cluster_set_df.cache()

In [16]:
while True:
    next_iteration_df = iteration_df.alias("c").join(
        cluster_set_df.alias("s"),
        F.col("c.cluster_types_set_id") == F.col("s.id")
    ).select(
        F.col("s.id"),
        F.col("s.cluster_type_id"),
        F.col("s.cluster_type_level"),
        F.col("s.cluster_types_set_id"),
        F.col("c.root")
    )
    
    new_rows_df = next_iteration_df.join(final_results_df, on="id", how="left_anti")
    
    new_rows_df = new_rows_df.cache()
    
    if new_rows_df.isEmpty():
        break
        
    final_results_df = final_results_df.unionByName(new_rows_df)
    
    iteration_df = new_rows_df

cluster_set_df.unpersist()

output_df = final_results_df.select(
    F.col("id").alias("property_id"),
    F.col("cluster_type_level").alias("order_number")
)

25/08/19 12:11:38 WARN BlockManager: Block rdd_5_0 already exists on this machine; not re-adding it
[Stage 14:===============================================>      (176 + 4) / 200]

## arr_cl_set

In [18]:
grouped_df = final_results_df.groupBy("root").agg(
    F.collect_list(F.col("cluster_type_id").cast("string")).alias("id_array"),
    F.collect_list(F.col("cluster_type_level").cast("string")).alias("level_array")
)

arr_cl_set_df = grouped_df.withColumn(
    "zipped_structs", F.arrays_zip(F.col("id_array"), F.col("level_array"))
).withColumn(
    "sorted_structs", F.array_sort(F.col("zipped_structs"))
).withColumn(
    "ct_lvl", F.expr("transform(sorted_structs, s -> array(s.id_array, s.level_array))")
).select("ct_lvl", "root")

## arr_prior

In [20]:
arr_prior_df = arr_cl_set_df.alias("ac").join(
    priority_df.alias("pr"),
    F.col("ac.root") == F.col("pr.cluster_types_set_id")
).select(
    F.col("ac.ct_lvl"),
    F.col("pr.metric_id"),
    F.col("pr.order_number")
)

## arr_prop

In [28]:
arr_prop_intermediate_df = property_df.alias("p").join(
    cluster_df.alias("c"),
    F.col("c.property_id") == F.col("p.id")
).where(
    F.col("p.is_exception") == False
).groupBy(
    "c.property_id",
    "p.metric_id"
).agg(
    F.collect_list(F.col("c.cluster_type_id").cast("string")).alias("id_array"),
    F.collect_list(F.col("c.level").cast("string")).alias("level_array") # Note: uses 'level' column
)

arr_prop_df = arr_prop_intermediate_df.withColumn(
    "sorted_structs", F.array_sort(F.arrays_zip(F.col("id_array"), F.col("level_array")))
).withColumn(
    "ct_lvl", F.expr("transform(sorted_structs, s -> array(s.id_array, s.level_array))")
).select(
    "property_id",
    "metric_id",
    "ct_lvl"
)

In [29]:
final_join_df = arr_prior_df.alias("p").join(
    arr_prop_df.alias("pr"),
    (F.col("p.ct_lvl") == F.col("pr.ct_lvl")) & (F.col("p.metric_id") == F.col("pr.metric_id"))
).select(
    F.col("pr.property_id"),
    F.col("p.order_number")
)

In [32]:
final_join_df.show()

+-----------+------------+
|property_id|order_number|
+-----------+------------+
|       4129|         190|
|       4129|         277|
|       4129|         133|
|       4129|          94|
|       4129|         238|
|       4129|          46|
|       4129|          91|
|       4129|         234|
|       4129|         286|
|       4129|         142|
|       4129|          44|
|       4129|         285|
|       4130|         190|
|       4130|         277|
|       4130|         133|
|       4130|          94|
|       4130|         238|
|       4130|          46|
|       4130|          91|
|       4130|         234|
+-----------+------------+
only showing top 20 rows


## Inserting data in params.promo_item_location_period_norm

In [39]:
promo_item_location_period_df = read_data("promo_item_location_period")
item_df = read_data("item")
item_expiration_period_df = read_data("item_expiration_period")
item_promo_df = read_data("item_promo")
promo_category_hierarchy_df = read_data("promo_category_hierarchy")
promo_period_df = read_data("promo_period")
store_df = read_data("store")
delivery_group_df = read_data("delivery_group")
delivery_group_assortment_df = read_data("delivery_group_assortment")

common_base_df = promo_item_location_period_df.alias("sc").join(
    item_df.alias("i"), F.col("i.item_uid") == F.col("sc.item_id")
).join(
    item_expiration_period_df.alias("ie"),
    F.col("i.total_shelf_life_day_qnty").between(F.col("ie.value_min"), F.col("ie.value_max"))
).join(
    item_promo_df.alias("ip"), F.col("ip.item_promo_uid") == F.col("sc.promo_id")
).join(
    promo_category_hierarchy_df.alias("ph"), F.col("ph.promo_category_uid") == F.col("ip.promo_category_uid")
).join(
    promo_period_df.alias("pd"),
    (F.datediff(F.col("ip.promo_end_dt"), F.col("ip.promo_begin_dt")) + 1).between(F.col("pd.value_min"), F.col("pd.value_max"))
)

common_base_df.cache()

store_branch_df = common_base_df.join(
    store_df.alias("s"), F.col("s.store_uid") == F.col("sc.location_id")
).select(
    F.col("sc.*"),
    F.col("i.item_category_level_0_uid"),
    F.col("i.item_category_level_1_uid"),
    F.col("i.item_category_level_2_uid"),
    F.col("i.item_category_level_3_uid"),
    F.col("i.item_category_level_4_uid"),
    F.col("ie.item_expiration_period_uid"),
    F.col("pd.promo_period_uid"),
    F.col("ph.promo_category_level_0_uid"),
    F.col("ph.promo_category_level_1_uid"),
    F.col("ph.promo_category_level_2_uid"),
    F.col("s.store_format_level_0_uid"),
    F.col("s.store_format_level_1_uid"),
    F.lit(None).cast("string").alias("delivery_group_uid") # Add null placeholder
)

delivery_branch_df = common_base_df.filter(
    F.col("sc.location_id") == -5042515683615240694
).join(
    delivery_group_df.alias("dg"), F.col("dg.destination_location_uid") == F.col("sc.location_id")
).join(
    delivery_group_assortment_df.alias("dga"),
    (F.col("dga.delivery_group_uid") == F.col("dg.delivery_group_uid")) &
    (F.col("dga.item_uid") == F.col("sc.item_id"))
).select(
    F.col("sc.*"),
    F.col("i.item_category_level_0_uid"),
    F.col("i.item_category_level_1_uid"),
    F.col("i.item_category_level_2_uid"),
    F.col("i.item_category_level_3_uid"),
    F.col("i.item_category_level_4_uid"),
    F.col("ie.item_expiration_period_uid"),
    F.col("pd.promo_period_uid"),
    F.col("ph.promo_category_level_0_uid"),
    F.col("ph.promo_category_level_1_uid"),
    F.col("ph.promo_category_level_2_uid"),
    F.lit(None).cast("string").alias("store_format_level_0_uid"),
    F.lit(None).cast("string").alias("store_format_level_1_uid"),
    F.col("dga.delivery_group_uid")
)

promo_item_location_period = store_branch_df.unionByName(delivery_branch_df)

common_base_df.unpersist()

write_data(table_name="promo_item_location_period_norm_spark", dataframe=promo_item_location_period)

25/08/20 17:05:48 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
                                                                                

## Insert into item_location_period_norm

In [40]:
item_location_period_df = read_data("item_location_period")
item_df = read_data("item")
item_expiration_period_df = read_data("item_expiration_period")
store_df = read_data("store")
delivery_group_df = read_data("delivery_group")
delivery_group_assortment_df = read_data("delivery_group_assortment")

common_base_df = item_location_period_df.alias("sc").join(
    item_df.alias("i"), F.col("i.item_uid") == F.col("sc.item_id")
).join(
    item_expiration_period_df.alias("ie"),
    F.col("i.total_shelf_life_day_qnty").between(F.col("ie.value_min"), F.col("ie.value_max"))
)

common_base_df.cache()

store_branch_df = common_base_df.join(
    store_df.alias("s"), F.col("s.store_uid") == F.col("sc.location_id")
).select(
    "sc.item_id",
    "sc.location_id",
    "sc.begin_dt",
    "sc.end_dt",
    "i.item_category_level_0_uid",
    "i.item_category_level_1_uid",
    "i.item_category_level_2_uid",
    "i.item_category_level_3_uid",
    "i.item_category_level_4_uid",
    "ie.item_expiration_period_uid",
    "s.store_format_level_0_uid",
    "s.store_format_level_1_uid",
    F.lit(None).cast("string").alias("delivery_group_uid")
)

delivery_branch_df = common_base_df.filter(
    F.col("sc.location_id") == -5042515683615240694
).join(
    delivery_group_df.alias("dg"), F.col("dg.destination_location_uid") == F.col("sc.location_id")
).join(
    delivery_group_assortment_df.alias("dga"),
    (F.col("dga.delivery_group_uid") == F.col("dg.delivery_group_uid")) & 
    (F.col("dga.item_uid") == F.col("sc.item_id"))
).select(
    "sc.item_id",
    "sc.location_id",
    "sc.begin_dt",
    "sc.end_dt",
    "i.item_category_level_0_uid",
    "i.item_category_level_1_uid",
    "i.item_category_level_2_uid",
    "i.item_category_level_3_uid",
    "i.item_category_level_4_uid",
    "ie.item_expiration_period_uid",
    F.lit(None).cast("string").alias("store_format_level_0_uid"),
    F.lit(None).cast("string").alias("store_format_level_1_uid"),
    "dga.delivery_group_uid"
)

item_location_period = store_branch_df.unionByName(delivery_branch_df)

common_base_df.unpersist()

write_data(table_name="item_location_period_norm_spark", dataframe=item_location_period)


## supplier_item_location_period

In [41]:
supplier_item_location_period_df = read_data("supplier_item_location_period")
item_df = read_data("item")
item_expiration_period_df = read_data("item_expiration_period")
store_df = read_data("store")
delivery_group_df = read_data("delivery_group")
delivery_group_assortment_df = read_data("delivery_group_assortment")

common_base_df = supplier_item_location_period_df.alias("sc").join(
    item_df.alias("i"), F.col("i.item_uid") == F.col("sc.item_id")
).join(
    item_expiration_period_df.alias("ie"),
    F.col("i.total_shelf_life_day_qnty").between(F.col("ie.value_min"), F.col("ie.value_max"))
)

common_base_df.cache()

store_branch_df = common_base_df.join(
    store_df.alias("s"), F.col("s.store_uid") == F.col("sc.location_id")
).select(
    "sc.item_id",
    "sc.location_id",
    "sc.supplier_id",
    "sc.begin_dt",
    "sc.end_dt",
    "i.item_category_level_0_uid",
    "i.item_category_level_1_uid",
    "i.item_category_level_2_uid",
    "i.item_category_level_3_uid",
    "i.item_category_level_4_uid",
    "ie.item_expiration_period_uid",
    "s.store_format_level_0_uid",
    "s.store_format_level_1_uid",
    F.lit(None).cast("string").alias("delivery_group_uid")
)
print("Store logic branch created.")

delivery_branch_df = common_base_df.filter(
    F.col("sc.location_id") == -5042515683615240694
).join(
    delivery_group_df.alias("dg"),
    (F.col("dg.destination_location_uid") == F.col("sc.location_id")) &
    (F.col("dg.source_location_uid") == F.col("sc.supplier_id"))
).join(
    delivery_group_assortment_df.alias("dga"),
    (F.col("dga.delivery_group_uid") == F.col("dg.delivery_group_uid")) &
    (F.col("dga.item_uid") == F.col("sc.item_id"))
).select(
    "sc.item_id",
    "sc.location_id",
    "sc.supplier_id",
    "sc.begin_dt",
    "sc.end_dt",
    "i.item_category_level_0_uid",
    "i.item_category_level_1_uid",
    "i.item_category_level_2_uid",
    "i.item_category_level_3_uid",
    "i.item_category_level_4_uid",
    "ie.item_expiration_period_uid",
    F.lit(None).cast("string").alias("store_format_level_0_uid"),
    F.lit(None).cast("string").alias("store_format_level_1_uid"),
    "dga.delivery_group_uid"
)

supplier_item_location_period = store_branch_df.unionByName(delivery_branch_df)

common_base_df.unpersist()

write_data(dataframe=supplier_item_location_period, table_name="supplier_item_location_period_norm_spark")


Store logic branch created.


## supplier_location_period_norm

In [45]:
supplier_location_period_df = read_data("supplier_location_period")
store_df = read_data("store")
delivery_group_df = read_data("delivery_group")

store_branch_df = supplier_location_period_df.alias("sc").join(
    store_df.alias("s"), F.col("s.store_uid") == F.col("sc.location_id")
).select(
    "sc.location_id",
    "sc.supplier_id",
    "sc.begin_dt",
    "sc.end_dt",
    "s.store_format_level_0_uid",
    "s.store_format_level_1_uid",
    F.lit(None).cast("string").alias("delivery_group_uid")
)

delivery_branch_df = supplier_location_period_df.alias("sc").filter(
    F.col("sc.location_id") == -5042515683615240694
).join(
    delivery_group_df.alias("dg"),
    (F.col("dg.destination_location_uid") == F.col("sc.location_id")) &
    (F.col("dg.source_location_uid") == F.col("sc.supplier_id"))
).select(
    "sc.location_id",
    "sc.supplier_id",
    "sc.begin_dt",
    "sc.end_dt",
    F.lit(None).cast("string").alias("store_format_level_0_uid"),
    F.lit(None).cast("string").alias("store_format_level_1_uid"),
    "dg.delivery_group_uid"
)

supplier_location_period_norm = store_branch_df.unionByName(delivery_branch_df)

write_data(dataframe=supplier_location_period_norm, table_name="supplier_location_period_norm_spark")
